<a href="https://colab.research.google.com/github/perezsam/DM19-Lab2-Homework/blob/master/kaggle_competition_logistic_regression_tweet_tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Load raw data from kaggle##

In [2]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"cssamuel","key":"428a235331998b968d7e4fdbfb08d70d"}'}

In [3]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 64 Dec 22 03:09 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!mkdir data

In [8]:
cd data

/content/data


In [9]:
# Copy the private data set locally.
!kaggle datasets download -d cssamuel/dmlab2

 97% 145M/149M [00:01<00:00, 116MB/s]
100% 149M/149M [00:01<00:00, 114MB/s]


In [10]:
ls

dmlab2.zip


In [11]:
pwd

'/content/data'

In [12]:
!unzip dmlab2.zip

Archive:  dmlab2.zip
  inflating: data_identification.csv  
  inflating: emotion.csv             
  inflating: sampleSubmission.csv    
  inflating: tweets_DM.json          


In [13]:
pwd

'/content/data'

In [14]:
cd ..

/content


In [15]:
ls

data/  kaggle.json  sample_data/


##Data preprocessing##

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
import json
from pandas.io.json import json_normalize

In [0]:
# read data files
raw_data=pd.read_json("./data/tweets_DM.json",lines=True)
tweets=json_normalize(data=raw_data['_source'])
identify=pd.read_csv("./data/data_identification.csv")
emotion=pd.read_csv("./data/emotion.csv")

In [19]:
# rename column names
tweets=tweets.rename(index=str,columns={"tweet.text":"text", "tweet.tweet_id":"tweet_id",
                                       "tweet.hashtags":"hashtags"})
# add identify tags to dataframe
tweets=pd.merge(tweets,identify, on="tweet_id")

#get training set and test set
train_df=tweets[tweets["identification"] == "train"]
test_df=tweets[tweets["identification"] == "test"]

#add emotion column
train_df=pd.merge(train_df,emotion, on="tweet_id")
test_df["emotion"]=""

#drop identification tags
train_df.drop(columns=["identification"],inplace=True)
test_df.drop(columns=["identification"],inplace=True)

#use tweet_id as index
train_df.set_index("tweet_id",inplace=True)
test_df.set_index("tweet_id",inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
!mkdir output

In [0]:
# save to pickle file
train_df.to_pickle("./output/train_df.pkl")
test_df.to_pickle("./output/test_df.pkl")

In [0]:
## load a pickle file
train_df = pd.read_pickle("./output/train_df.pkl")
test_df = pd.read_pickle("./output/test_df.pkl")


##Logistic Regression##

In [0]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [0]:
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
s1 = '@remy: This is waaaaayyyy too much for you!!!!!!'
tknzr.tokenize(s1)

[':', 'This', 'is', 'waaayyy', 'too', 'much', 'for', 'you', '!', '!', '!']

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(preserve_case=False)
tfidf = TfidfVectorizer(max_features=80000, stop_words='english',
                                     tokenizer=tknzr.tokenize)

# fitting
tfidf.fit(train_df['text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=80000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f18a99b9ac8>>,
                use_idf=True, vocabulary=None)

In [0]:
# transforming training sets
X_train = tfidf.transform(train_df['text'])
X_train.shape

(1455563, 80000)

In [0]:
# transforming testing sets
X_test = tfidf.transform(test_df['text'])
X_test.shape

(411972, 80000)

In [0]:
# set pointers
y_train = train_df['emotion']
y_test = test_df['emotion']

In [0]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=10,n_jobs=-1,max_iter=1000)
lr.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
pred_result_lr = lr.predict(X_test)
pred_result_lr.shape

(411972,)

In [0]:
# save the result
test_df['emotion']=pred_result_lr
test_df.drop(columns=['hashtags','text'],inplace=True)
test_df.index.rename('id',inplace=True)
test_df.columns=['emotion']
test_df.to_csv('logistic_regression_C10_tfidf_80000_A.csv')